In [ ]:
!pip install pynytimes --quiet
# https://github.com/michadenheijer/pynytimes

In [35]:
from pynytimes import NYTAPI
import pandas as pd
import glob
from ast import literal_eval
import matplotlib.pyplot as plt
plt.style.use('dark_background')
pd.options.display.max_colwidth = 100
import nltk
from nltk.corpus import stopwords
from datetime import datetime, timedelta
import ast

In [37]:
nyt = NYTAPI("owjL3q1wD8x7tvdC8UGXBBLKsvaA6eoV", parse_dates=True)

In [39]:
# Calculate the start and end date (past 100 days)
start_date = (datetime.today() - timedelta(days=100)).date()
end_date = datetime.today().date()

# Define multiple queries related to UAP, UFOs, Aliens, and Drones
search_queries = ["Drones", "Drone sighting"]

# Fetch articles using the queries and correct date format
articles = nyt.article_search(
    query=" OR ".join(search_queries),  # Combine queries with OR
    dates={"begin": start_date, "end": end_date},  # Use a dictionary for dates
    results=10  # Adjust as needed
)
df = pd.DataFrame.from_dict(articles)

In [41]:
df.head()

,abstract,web_url,snippet,lead_paragraph,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,print_section,print_page,subsection_name
0,The governor made the announcement on Friday as sightings spread across New York and New Jersey....,https://www.nytimes.com/2024/12/13/nyregion/nj-drones-hochul-east-coast.html,The governor made the announcement on Friday as sightings spread across New York and New Jersey....,"As reports of drone sightings continued to spread across New Jersey and New York, Gov. Kathy Hoc...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'imag...","{'main': 'Hochul Launches Investigation Into Drone Sightings in New York', 'kicker': None, 'cont...","[{'name': 'subject', 'value': 'Drones (Pilotless Planes)', 'rank': 1, 'major': 'N'}, {'name': 's...",2024-12-13 19:15:17+00:00,article,Metro,New York,"{'original': 'By Alyce McFadden', 'person': [{'firstname': 'Alyce', 'middlename': None, 'lastnam...",News,nyt://article/37b08bdd-2403-56d5-ab3d-c2ef4f3ba191,422,nyt://article/37b08bdd-2403-56d5-ab3d-c2ef4f3ba191,NaN,NaN,NaN
1,Drones have been reported in at least 10 counties of the state. But a White House spokesman sugg...,https://www.nytimes.com/2024/12/11/nyregion/new-jersey-drones.html,Drones have been reported in at least 10 counties of the state. But a White House spokesman sugg...,Dozens of reported drone sightings in the skies above New Jersey have alarmed lawmakers and loca...,The New York Times,[],"{'main': 'Are Those Drones Over New Jersey? Despite Sightings, U.S. Is Skeptical.', 'kicker': No...","[{'name': 'subject', 'value': 'Drones (Pilotless Planes)', 'rank': 1, 'major': 'N'}, {'name': 'g...",2024-12-11 20:51:37+00:00,article,Metro,New York,"{'original': 'By Alyce McFadden', 'person': [{'firstname': 'Alyce', 'middlename': None, 'lastnam...",News,nyt://article/71566340-cf2e-565e-9595-e3b6270844a7,1039,nyt://article/71566340-cf2e-565e-9595-e3b6270844a7,A,22,NaN
2,"“‘Nothing nefarious, apparently’ doesn’t exactly inspire confidence,” Stephen Colbert said of Pr...",https://www.nytimes.com/2024/12/19/arts/television/late-night-drones.html,"“‘Nothing nefarious, apparently’ doesn’t exactly inspire confidence,” Stephen Colbert said of Pr...","Welcome to Best of Late Night, a rundown of the previous night’s highlights that lets you sleep ...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'imag...","{'main': 'Late Night Continues to Drone On About Sightings in the Sky', 'kicker': 'Best of Late ...","[{'name': 'subject', 'value': 'Television', 'rank': 1, 'major': 'N'}, {'name': 'persons', 'value...",2024-12-19 08:54:03+00:00,article,Culture,Arts,"{'original': 'By Trish Bendix', 'person': [{'firstname': 'Trish', 'middlename': None, 'lastname'...",News,nyt://article/c5ba6219-3e78-5585-bce8-4c727f2773a0,520,nyt://article/c5ba6219-3e78-5585-bce8-4c727f2773a0,NaN,NaN,Television
3,"In the Garden State, where the rash of sightings started a month ago, residents are looking to t...",https://www.nytimes.com/2024/12/14/nyregion/drone-sightings-nj.html,"In the Garden State, where the rash of sightings started a month ago, residents are looking to t...","The day after Thanksgiving, Susan and Lorelai Woodruff saw approximately 10 brightly lit objects...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'imag...","{'main': 'Weeks of Drone Sightings Leave New Jersey on Edge', 'kicker': None, 'content_kicker': ...","[{'name': 'glocations', 'value': 'New Jersey', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'va...",2024-12-15 00:18:10+00:00,article,Metro,New York,"{'original': 'By Alyce McFadden and Nate Schweber', 'person': [{'firstname': 'Alyce', 'middlenam...",News,nyt://article/b630dd82-8e86-5388-acc4-7cab5a4cce7f,914,nyt://article/b630dd82-8e86-5388-acc4-7cab5a4cce

In [47]:
df[['keywords']].head(1)

,keywords
0,"[{'name': 'subject', 'value': 'Drones (Pilotless Planes)', 'rank': 1, 'major': 'N'}, {'name': 's..."


In [45]:
df["title"] = df["headline"].apply(lambda x: x.get("main", "").strip() if isinstance(x, dict) else x)


In [49]:
import ast

def extract_keywords(keyword_list):
    if isinstance(keyword_list, str):  # Ensure it's a string before parsing
        try:
            keyword_list = ast.literal_eval(keyword_list)  # Convert string representation of list to actual list
        except:
            return None  # Return None if conversion fails
    
    if isinstance(keyword_list, list):
        return ", ".join([kw["value"] for kw in keyword_list if "value" in kw])  # Extract values and join them

    return None  # Return None if it's not a list

# Apply transformation to extract only the keyword values
df["keywords"] = df["keywords"].apply(extract_keywords)

# Display updated column
df["keywords"].head()


0    Drones (Pilotless Planes), Governors (US), Hochul, Kathleen C, Hogan, Lawrence J, MORRIS COUNTY ...
1    Drones (Pilotless Planes), New Jersey, Social Media, Federal Aviation Administration, Federal Bu...
2    Television, Colbert, Stephen, Late Show with Stephen Colbert (TV Program), Late Show With David ...
3                                                    New Jersey, Drones (Pilotless Planes), Social Media
4    Drones (Pilotless Planes), Northeastern States (US), United States Politics and Government, Mili...
Name: keywords, dtype: object

In [51]:
df.head()

,abstract,web_url,snippet,lead_paragraph,source,multimedia,headline,keywords,pub_date,document_type,...,section_name,byline,type_of_material,_id,word_count,uri,print_section,print_page,subsection_name,title
0,The governor made the announcement on Friday as sightings spread across New York and New Jersey....,https://www.nytimes.com/2024/12/13/nyregion/nj-drones-hochul-east-coast.html,The governor made the announcement on Friday as sightings spread across New York and New Jersey....,"As reports of drone sightings continued to spread across New Jersey and New York, Gov. Kathy Hoc...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'imag...","{'main': 'Hochul Launches Investigation Into Drone Sightings in New York', 'kicker': None, 'cont...","Drones (Pilotless Planes), Governors (US), Hochul, Kathleen C, Hogan, Lawrence J, MORRIS COUNTY ...",2024-12-13 19:15:17+00:00,article,...,New York,"{'original': 'By Alyce McFadden', 'person': [{'firstname': 'Alyce', 'middlename': None, 'lastnam...",News,nyt://article/37b08bdd-2403-56d5-ab3d-c2ef4f3ba191,422,nyt://article/37b08bdd-2403-56d5-ab3d-c2ef4f3ba191,NaN,NaN,NaN,Hochul Launches Investigation Into Drone Sightings in New York
1,Drones have been reported in at least 10 counties of the state. But a White House spokesman sugg...,https://www.nytimes.com/2024/12/11/nyregion/new-jersey-drones.html,Drones have been reported in at least 10 counties of the state. But a White House spokesman sugg...,Dozens of reported drone sightings in the skies above New Jersey have alarmed lawmakers and loca...,The New York Times,[],"{'main': 'Are Those Drones Over New Jersey? Despite Sightings, U.S. Is Skeptical.', 'kicker': No...","Drones (Pilotless Planes), New Jersey, Social Media, Federal Aviation Administration, Federal Bu...",2024-12-11 20:51:37+00:00,article,...,New York,"{'original': 'By Alyce McFadden', 'person': [{'firstname': 'Alyce', 'middlename': None, 'lastnam...",News,nyt://article/71566340-cf2e-565e-9595-e3b6270844a7,1039,nyt://article/71566340-cf2e-565e-9595-e3b6270844a7,A,22,NaN,"Are Those Drones Over New Jersey? Despite Sightings, U.S. Is Skeptical."
2,"“‘Nothing nefarious, apparently’ doesn’t exactly inspire confidence,” Stephen Colbert said of Pr...",https://www.nytimes.com/2024/12/19/arts/television/late-night-drones.html,"“‘Nothing nefarious, apparently’ doesn’t exactly inspire confidence,” Stephen Colbert said of Pr...","Welcome to Best of Late Night, a rundown of the previous night’s highlights that lets you sleep ...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'imag...","{'main': 'Late Night Continues to Drone On About Sightings in the Sky', 'kicker': 'Best of Late ...","Television, Colbert, Stephen, Late Show with Stephen Colbert (TV Program), Late Show With David ...",2024-12-19 08:54:03+00:00,article,...,Arts,"{'original': 'By Trish Bendix', 'person': [{'firstname': 'Trish', 'middlename': None, 'lastname'...",News,nyt://article/c5ba6219-3e78-5585-bce8-4c727f2773a0,520,nyt://article/c5ba6219-3e78-5585-bce8-4c727f2773a0,NaN,NaN,Television,Late Night Continues to Drone On About Sightings in the Sky
3,"In the Garden State, where the rash of sightings started a month ago, residents are looking to t...",https://www.nytimes.com/2024/12/14/nyregion/drone-sightings-nj.html,"In the Garden State, where the rash of sightings started a month ago, residents are looking to t...","The day after Thanksgiving, Susan and Lorelai Woodruff saw approximately 10 brightly lit objects...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'imag...","{'main': 'Weeks of Drone Sightings Leave New Jersey on Edge', 'kicker': None, 'content_kicker': ...","New Jersey, Drones (Pilotless Planes), Social Media",2024-12-15 00:18:10+00:00,article,...,New York,"{'original': 'By Alyce McFadden and Nate Schweber', 'person': [{'first

In [ ]:
df.to_csv('dronearticles.csv')

In [ ]:
keyword_counts = pd.Series(x.lower() for l in df['des_facet'] for x in l).value_counts(ascending=False)
keyword_counts = keyword_counts[keyword_counts >= 2]
len(keyword_counts)

In [ ]:
keyword_counts.head(25)

In [ ]:
#pd_keywords = keyword_counts.to_frame()

pd_keywords = pd.DataFrame({'keywords':keyword_counts.index, 'frequencies':keyword_counts.values})

In [ ]:
pd_keywords.head()

In [ ]:
#keyword_counts_list = list(keyword_counts.keys())
#frequencies = []
#for keyword in deaf_keywords:
#    index = keyword_counts_list.index(keyword)
#    frequencies.append(keyword_counts.iloc[index])
    
plt.figure(figsize=(10, 5))
plt.barh(pd_keywords['keywords'], pd_keywords['frequencies'])
plt.title('NYT keyword frequency')
plt.show()